In [ ]:
import os
import json
import cv2


In [ ]:
def image_upside_down_flipping(image, bbox):

    flipped_image = cv2.flip(image, 0)
    x_min, y_min, width, height = bbox

    y_min = int((640 - y_min - height) / 2)
    x_min = int(x_min / 2)

    width = int(width / 2)
    height = int(height / 2)

    re_bbox = [x_min, y_min, width, height]
    return flipped_image, re_bbox

In [ ]:
def image_leftright_down_flipping(image, bbox):

    flipped_image = cv2.flip(image, 1)
    x_min, y_min, width, height = bbox

    x_min = int((640 - x_min - width) / 2)
    y_min = int (y_min / 2)
    width = int(width / 2)
    height = int(height / 2)

    re_bbox = [x_min, y_min, width, height]
    return flipped_image, re_bbox

In [ ]:
def both_down_flipping(image, bbox):

    image = cv2.flip(image, 0)
    flipped_image = cv2.flip(image, 1)
    x_min, y_min, width, height = bbox

    y_min = int((640 - y_min - height) / 2)
    x_min = int((640 - x_min - width) / 2)
    width = int(width / 2)
    height = int(height / 2)

    re_bbox = [x_min, y_min, width, height]
    return flipped_image, re_bbox

In [ ]:
def image_augmentation(input_image_path, output_image_path, image_annotation_path, output_image_annotation_path):

    with open(image_annotation_path, 'r') as file:
        content = file.read()

    data = json.loads(content)
    filenames = os.listdir(input_image_path)
    output_folder = output_image_path
    filenames.sort()
    annotations = []
    number = 0
    origin_image_num = 0
    for filename in filenames:
        image = cv2.imread(f"{input_image_path}/{filename}")
        image = cv2.resize(image, (320, 320))
        print(filename)
        x_min, y_min, width, height = data[origin_image_num]["bbox"]
        y_min = int(y_min / 2)
        x_min = int(x_min / 2)
        width = int(width / 2)
        height = int(height / 2)
        re_bbox = [x_min, y_min, width, height]
        annotation = {"id": number, "image_id": number, "bbox": re_bbox, "area": int(data[origin_image_num]["area"] / 4), "category_id": data[origin_image_num]["category_id"]}
        annotations.append(annotation)
        output_path = f"{output_folder}/{number}.jpg"
        cv2.imwrite(output_path, image)
        number += 1
        print(f"image {number}.jpg is already process")


        flipped_image, bbox = image_upside_down_flipping(image, data[origin_image_num]["bbox"])
        annotation = {"id": number, "image_id": number, "bbox": bbox, "area": int(data[origin_image_num]["area"] / 4), "category_id": data[origin_image_num]["category_id"]}
        annotations.append(annotation)
        output_path = f"{output_folder}/{number}.jpg"
        cv2.imwrite(output_path, flipped_image)
        number += 1
        print(f"image {number}.jpg is already process")


        flipped_image, bbox = image_leftright_down_flipping(image, data[origin_image_num]["bbox"])
        annotation = {"id": number, "image_id": number, "bbox": bbox, "area": int(data[origin_image_num]["area"] / 4), "category_id": data[origin_image_num]["category_id"]}
        annotations.append(annotation)
        output_path = f"{output_folder}/{number}.jpg"
        cv2.imwrite(output_path, flipped_image)
        number += 1
        print(f"image {number}.jpg is already process")


        flipped_image, bbox = both_down_flipping(image, data[origin_image_num]["bbox"])
        annotation = {"id": number, "image_id": number, "bbox": bbox, "area": int(data[origin_image_num]["area"] / 4), "category_id": data[origin_image_num]["category_id"]}
        annotations.append(annotation)
        output_path = f"{output_folder}/{number}.jpg"
        cv2.imwrite(output_path, flipped_image)
        number += 1
        print(f"image {number}.jpg is already process")


        origin_image_num += 1

    with open(output_image_annotation_path, "w") as file:
        json.dump(annotations, file)





In [ ]:
def folder_create(filename):
    folder_path = filename

    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print("Folder created successfully.")
    else:
        print("Folder already exists.")


In [ ]:
folder_name = ["../train/augmentation/", "../train/pre_augmentation/", "../valid/augmentation/", "../valid/pre_augmentation/"]

for i in folder_name:
    folder_create(i)

image_augmentation("../train/train/", "../train/augmentation/", "../train_annotations", "../train_annotations_augmentation")

image_augmentation("../train/preprocess/", "../train/pre_augmentation/", "../train_annotations", "../train_annotations_augmentation")

image_augmentation("../valid/valid/", "../valid/augmentation/", "../valid_annotations", "../valid_annotations_augmentation")

image_augmentation("../valid/preprocess/", "../valid/pre_augmentation/", "../valid_annotations", "../valid_annotations_augmentation")